In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from utils import get_df, analyze_results, plot_accuracy

In [3]:
path = "ray/criteo/bias_varying_workload_size"
df = analyze_results(path, "bias", parallelize=False)
for destination in df.groupby(['destination']).destination.unique():
    advertiser = df[df['destination'].isin(destination)]
    plot_accuracy(advertiser)

In [4]:
path = "ray/criteo/bias_varying_workload_size"
budget_df = analyze_results(path, "budget")
budget_df

{'32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404, '33': 0.0230258509299404, '34': 0.0230258509299404, '35': 0.0230258509299404, '36': 0.0230258509299404, '37': 0.0230258509299404, '38': 0.0230258509299404, '39': 0.0230258509299404, '40': 0.0230258509299404, '41': 0.0230258509299404, '42': 0.0230258509299404}

{'32': 0.0230258509299404}


{'32': 0.0230258509299404}
{'38': 0.0888685871475895, '39': 0.0888685871475895, '40': 0.0888685871475895, '41': 0.0888685871475895, '42': 0.0888685871475895, '43': 0.0888685871475895, '44': 0.0888685871475895, '45': 0.0888685871475895, '46': 0.0888685871475895}{'32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404, '33': 0.0230258509299404, '34': 0.0230258509299404, '35': 0.0230258509299404, '36':

Process Process-4:



{'32': 0.0230258509299404}
{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404, '33': 0.0230258509299404, '34': 0.0230258509299404, '35': 0.0230258509299404, '36': 0.0230258509299404, '37': 0.0230258509299404, '38': 0.0230258509299404}{'32': 0.0230258509299404}

{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404, '33': 0.0230258509299404, '34': 0.0230258509299404, '35': 0.0230258509299404, '36': 0.0230258509299404, '37': 0.0230258509299404, '38': 0.0230258509299404, '39': 0.0230258509299404, '40': 0.0230258509299404, '41': 0.0230258509299404, '42': 0.0230258509299404}

Traceback (most recent call last):


{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}{'32': 0.0230258509299404}{'34': 0.0230258509299404, '35': 0.0230258509299404, '36': 0.0230258509299404, '37': 0.0230258509299404, '38': 0.0230258509299404, '39': 0.0230258509299404, '40': 0.0230258509299404, '41': 0.0230258509299404, '42': 0.0230258509299404, '43': 0.0230258509299404}


...


{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}
{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}{'32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}


  File "/mydata/cookiemonster/notebooks/utils.py", line 72, in get_budget_logs
    for epoch, budget_consumed in budget_per_epoch.items():





{'32': 0.0230258509299404}{'32': 0.0230258509299404}



AttributeError: 'float' object has no attribute 'items'


{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}
{'32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}
{'32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}{'32': 0.0230258509299404}



{'32': 0.0230258509299404}


{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '31': 0.0230258509299404, '32': 0.0230258509299404}{'27': 0.0230258509299404, '28': 0.0230258509299404, '29': 0.0230258509299404, '30': 0.0230258509299404, '3